Question -3
How does the price changes for neighbourhood throughout the year?
1. Plotting line graph to see the variation (Use average price of the same day to calculate the data)


In [290]:
# Imports
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import folium
import json
import re
from collections import Counter
from scipy import stats
import geopandas as gpd

In [291]:
listings_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/listings.csv'
listings_df = pd.read_csv(listings_filepath)

In [292]:
calendar_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/calendar.csv'
calendar_df = pd.read_csv(calendar_filepath)

In [293]:
neighbourhoods_filepath = '/Users/abhayrakeshdeshmukhmaurya/Library/CloudStorage/OneDrive-UniversityofEdinburgh/CS/University/Year 2/Data Science/CW3/AirBnb/datasets/neighbourhoods.csv'
neighbourhoods_df = pd.read_csv(neighbourhoods_filepath)

In [294]:
listings_for_calendar = listings_df.loc[:, ['id', 'neighbourhood_cleansed', 'latitude', 'longitude']]

In [295]:
calendar_df['listing_id'].nunique()

7389

In [296]:
# calendar_df = calendar_df['price'].dropna()
# calendar_df['price'] = calendar_df['price'].str.replace('$', '')  # Remove dollar sign
# calendar_df['price'] = calendar_df['price'].str.replace(',', '')  # Remove comma separators (if present)
# calendar_df['price'] = calendar_df['price'].astype(float).astype(int)  # Convert to float and then to int

calendar_df['price'] = calendar_df['price'].str.replace(',', '',regex=True).str.replace('$', '',regex=True).astype('float')


# calendar_df['date'] = pd.to_datetime(calendar_df['date'])
# # Group the data by date and calculate the mean price for each date
# avg_price_by_date = calendar_df.groupby('date')['price'].mean()

In [297]:
Q1 = calendar_df['price'].quantile(0.25)
Q3 = calendar_df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outlier detection
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

# Identify and remove outliers
calendar_df = calendar_df[(calendar_df['price'] >= lower_bound) & (calendar_df['price'] <= upper_bound)]

In [298]:
calendar_df['price']

0          100.0
1          115.0
2          115.0
3          115.0
4          115.0
           ...  
2696631     44.0
2696632     44.0
2696633     44.0
2696634     44.0
2696635     44.0
Name: price, Length: 2483018, dtype: float64

In [299]:
pivot_df = calendar_df.pivot(index='listing_id', columns='date', values='price')
pivot_df.to_csv('../datasets/pivot_df.csv')
pivot_df['2022-12-18']


listing_id
15420                 115.0
24288                  73.0
38628                  55.0
44009                  45.0
44552                  44.0
                      ...  
782136800346144859    350.0
782448596836883109    100.0
782474793883477230     35.0
782510024659134909     80.0
782521844972923104     95.0
Name: 2022-12-18, Length: 7162, dtype: float64

In [300]:

# Load the dataset
# quality_df = pd.read_csv('quality_data.csv')

# Calculate the IQR
# Q1 = quality_df['price'].quantile(0.25)
# Q3 = quality_df['price'].quantile(0.75)
# IQR = Q3 - Q1

# # Define the lower and upper bounds for outlier detection
# lower_bound = Q1 - 1.5*IQR
# upper_bound = Q3 + 1.5*IQR

# # Identify and remove outliers
# quality_df = quality_df[(quality_df['price'] >= lower_bound) & (quality_df['price'] <= upper_bound)]

In [301]:
# pivot_df.dropna(inplace=True)
pivot_df.fillna(pivot_df.median()) 


date,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-07,2023-12-08,2023-12-09,2023-12-10,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16
listing_id,,,,,,,,,,,,,,,,,,,,,
15420,100.0,115.0,115.0,115.0,115.0,120.0,120.0,120.0,130.0,130.0,...,120.0,120.0,120.0,120.0,120.0,120.0,120.0,120.0,120.0,135.5
24288,110.0,79.0,73.0,65.0,65.0,65.0,65.0,67.0,65.0,65.0,...,94.0,94.0,94.0,94.0,93.0,93.0,93.0,94.0,94.0,94.0
38628,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,...,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,135.5
44009,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,135.5
44552,110.0,42.0,44.0,44.0,44.0,44.0,44.0,42.0,42.0,56.0,...,39.0,42.0,42.0,39.0,39.0,39.0,39.0,39.0,42.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782136800346144859,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,295.0,350.0,...,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,135.5
782448596836883109,110.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
782474793883477230,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,135.5


In [302]:
# pivot_df.dropna(inplace=True)
pivot_df

date,2022-12-16,2022-12-17,2022-12-18,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-24,2022-12-25,...,2023-12-07,2023-12-08,2023-12-09,2023-12-10,2023-12-11,2023-12-12,2023-12-13,2023-12-14,2023-12-15,2023-12-16
listing_id,,,,,,,,,,,,,,,,,,,,,
1293370,65.0,65.0,65.0,58.0,58.0,58.0,58.0,65.0,65.0,55.0,...,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0
13315322,150.0,150.0,150.0,150.0,150.0,110.0,150.0,150.0,150.0,130.0,...,89.0,110.0,110.0,89.0,89.0,89.0,89.0,89.0,110.0,110.0
49364260,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,65.0,...,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
617953703959276311,110.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,70.0,70.0,...,88.0,90.0,90.0,90.0,88.0,88.0,88.0,88.0,90.0,91.0
